**STACK INSTRUCTIONS** 
- R6 will store "going" addresses while going to push/pop
- R7 will store "return" addresses while going to push/pop
- R5 will be used in push/pop to point to top of stack, so won't be saved

To call a subroutine: 
-     LEA R6, [SUBROUTINE LABEL]
-     JSR PUSH

Before returning from a subroutine: 
-     LEA R0, POP
-     JMP R0

Path: 
- Store address of subroutine in R6
- JSR to push, which stores starting point in R7
- Push does JMP R6 to get to subroutine
- Subroutine JMPs to pop
- Pop RETs to starting point 

Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

**ROOM/STATE DATA FORMAT**

(subject to changes?)

Every room/state stored in memory has a label pointing to the beginning of its data block, and variables stored at regular distances relative to that top. 

For all rooms:
- Label + 0: which quality (HP/stats) the room changes (0 for none) [Note: Can this field also be used to change variables, e.g. whether you have a sword?]
- Label + 1: amount of quality change
- Label + 2: type of state (0 for user-input directed, 1 for dice-directed, 2 for variable-directed)

Then if user-input room: 
- Label + 3, 4, 5: addresses of destinations for 1, 2, and 3 inputs
- Label + 6: Placeholder (so description string is same distance from start as with other types)

Or if dice room: 
- Label + 3: Stat to check
- Label + 4: Difficulty of check
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if variable room:
- Label + 3: Address of variable to check 
- Label + 4: Value of that variable that counts as a success
- Label + 5: Destination if success
- Label + 6: Destination if failure

Then for all rooms: 
- Label + 7: Description/options string in compressed format

In [2]:
.ORIG x3000
BRnzp LOOP_START

;;-----------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;;-----------------------------------------------------------------
;; Main loop variables 
LOC: .FILL ROOM_UINPUT
NAME: .STRINGC "Ruby"
HP: .FILL #60
;;-----------------------------------------------------------------
;; Main game loop
LOOP_START: 

    ;; Todo: Skip to endgame if your HP is 0
    ;; Todo: Print name and HP?

    LD R1, LOC     ;; Load current state's address into R1 so we can access variables relative to it
    
    ADD R0, R1, #7 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description

    LDR R2, R1, #0 ;; Load R1 with which_quality variable
    BRz NO_CHANGE  ;; If which_quality is zero, skip stat change
    ADD R2, R2, #-1 
    BRnp NOT_HP    ;; If which_quality is one, change HP by appropriate amount [Note: this should support other stats changing too lol]
    LDR R2, R1, #1 ;; Load change_amt into R1
    LD R3, HP      ;; Load stat to change (HP) into R2
    ADD R3, R3, R2 ;; Add change_amt to HP in R2
    ST R3, HP      ;; Store back in HP
NOT_HP:
NO_CHANGE:
    LDR R2, R1, #2 ;; Load room_type into R1
    ADD R2, R2, #-1 
    BRn UINPUT     ;; Break to code for appropriate room type
    ;;BRz DICE
    ;;BRp VARIABLE   

UINPUT:            ;; For user-directed rooms, use subroutine that reads & parses number from 1-3
    LEA R6, CHOICE
    JSR PUSH

;; Code for dice rooms and variable rooms....... coming soon???

BRnzp LOOP_START   ;; Repeat main game loop
HALT               ;; Right now this halt is never reached bc it loops forever LMAO

;; ------------------------------------------------------------------
;; Subroutine to get a number from 1-3 from the user
;; Copied from Anita's code in printing_text

CHOICE:   
          IN
          LD R2, LOC
          LD R1, ASCII     ;; Load inverted ASCII conversion constant into R1
          ADD R0, R0, R1   ;; Add it to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CH1 ;; user chose 1
          BRz CH2 ;; user chose 2
          BRp CH3 ;; user chose 3
          
CH1:      ADD R2, R2, #3
          BRnzp DONE
CH2:      ADD R2, R2, #4
          BRnzp DONE
CH3:      ADD R2, R2, #5
          BRnzp DONE
          
DONE:     LD LOC, R2 ;; Load the data at R2, aka the address of the #[choice] room, into LOC, moving player there
          LEA R0, POP
          JMP R0

;; Data
ASCII:   .FILL x-30

;; ------------------------------------------------------------------
;; Rooms data
ROOM_UINPUT: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT ;; Three choices all lead back here
    .FILL #0          ;; Placeholder
    .STRINGC "Here we are in an example room. Enter 1 to jump in place, 2 to do a jumping jack, or 3 to spin in a circle\n"
    
ROOM_RANDOM: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #1 ;; Stat for dice roll: ???? 
    .FILL #1 ;; Difficulty of check
    .FILL ROOM_RANDOM ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    .STRINGC "This room decides where to send you based on a dice roll. But it still only sends you back here."
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL #0 ;; VAR_ADDRESS ;; What variable to check
    .FILL #0 ;; VALUE       ;; What value counts as a success
    .FILL ROOM_VARIABLE ;; Destination if success
    .FILL ROOM_VARIABLE ;; Destination if failure
    .STRINGC "This room checks the value of a variable to determine where to send you." 
    
.END

Assembled! Use %dis or %dump to examine; use %exe to run.
